# Descrizione del problema
Il problema riguarda lo **scheduling**. Sono dati $G$ ingegneri che lavorono per $L$ giorni. La realtà è composta da 3 elementi:
* **Features**: sono le funzionalità implementate nei prodotti;
* **Services**: sono i programmi che sono in esecuzione sui server;
* **Binaries**: modellano un server. Ogni *binary* può avere in esecuzione più servizi;

## Relazioni tra le entità
Una **Feature** è contraddistinta dal numero di utenti che riesce a soddisfare e da un insieme di servizi in cui deve essere implementata. Una **Feature** è disponibile ai clienti quando viene implementata in ogni servizio da cui dipende.
Un **Service** è presente su **uno e un solo binary**, ma su ogni **binary** è possibile inserire più **Services**.

### Lettura input da file
Come prima cosa vengono letti i dati input e si effettua il parsing della prima riga per avere la dimensione del problema.

In [91]:
import random
import pathlib
from dataclasses import dataclass
from typing import List, Tuple, Dict, Set

fname = "data/f.in"

# lettura dei dati dal file di input
with open(fname, "r") as f:
    lines = f.readlines()
    
# parsing della prima riga: dimensione del problema
L, G, S, B, F, N = [int(k) for k in lines[0].strip().split(" ")]

# modellazione del sistema

score: int = 0
daily_schedule: List[Tuple[int, int]] = [] # giorno, binary

@dataclass
class Binary:
    id: int
    services: List[int]
    working_on: int
    locked: bool
binaries: List[Binary] =  [None] * B
service_to_binaries: Dict[str, int] = {}

@dataclass
class Service:
    id: int
    name: str
    binary: int
    impl_features: List[str]

services: List[Service] = []

@dataclass
class Feature:
    idx: int
    name: str
    difficulty: int
    users: int
    dep_services: List[List]
    completed: bool
    binaries: Set[int]
    day_completed: int
    def populate_binaries(self):
        for (service, done) in self.dep_services:
            b = service_to_binaries[service]
            self.binaries.add(b)


features: List[Feature] = []

@dataclass
class Engineer:
    id: int
    avail: int
    ops: List[Tuple[str, List]]

    def add_op(self, op_name: str, day: int, duration: int, values: List[int]):
        if day > self.avail:
            self.ops.append(("wait", [day - self.avail]))

        affected_binaries: List[int] = []
        if op_name == "move":
            service = services[values[0]]
            b1 = service_to_binaries[service.name]
            affected_binaries.append(b1)
            affected_binaries.append(values[1])
        elif op_name == "impl":
            affected_binaries.append(values[1])
        elif op_name not in ["new", "wait"]:
            raise Exception(f"operation {op_name} not supported")

        # controllo se un binary coinvolto nell'operazione è bloccato
        # se si aggiungo una wait pari al massimo di quelli schedulati
        locked = [b for b in affected_binaries if binaries[b].locked]
        skip = []
        for l in locked:
            for ds in daily_schedule:
                if ds[1] == l:
                    skip.append(ds[0])
        else:
            if len(skip) > 0:
                self.ops.append(("wait", [day + max(skip)]))

        self.avail += day + duration
        self.ops.append((op_name, values))
engineers: List[Engineer] = []

for i in range(G):
    engineers.append(Engineer(i, 0, []))

idx = 0
for (service, binary) in [line.strip().split() for line in lines[1:S+1]]:
    bid = int(binary)
    b = binaries[bid]
    if b is None:
        b = Binary(bid, [idx], 0, False)
        binaries[bid] = b
    else:
        b.services.append(idx)

    s = Service(idx, service, bid, [])
    idx+=1
    services.append(s)
    service_to_binaries[service] = bid

idx = 0
for i, feature in enumerate(lines[1+S:]):
    # leggo solo le righe pari
    if i % 2 ==0:
        feature, num_services, difficulty, users = feature.strip().split(" ")
        s = [[s, False] for s in lines[1+S+1+i].split()]
        f = Feature(idx, feature, int(difficulty), int(users), s, False, set(), 0)
        f.populate_binaries()
        features.append(f)
        idx += 1

print(binaries)

def implement_feature(e: int, day: int, f: int, b: int) -> int:
    assert(b >= 0 and b < B)
    assert(day >=0 and day < L)
    assert(f >= 0 and f < F)
    assert(e >= 0 and e < G)
    global score
    engineer = engineers[e]
    feature = features[f]
    
    duration  = feature.difficulty + len(binaries[b].services) + binaries[b].working_on
    for service in feature.dep_services:
        s_name = service[0]
        done = service[1]
        if service_to_binaries[s_name] == b and not done:
            service[1] = True
    
    for (service, done) in feature.dep_services:
        if not done:
            break
    else:
        if not feature.completed:
            feature.completed = True
            feature.day_completed = day + duration
            # print("DEBUG: completed", feature, "multiply", 1 if  L-(day + duration) >= 0 else 0  )
            # print("DEBUG: remaining", len(features))
            score += feature.users * max(0, L-(day + duration)) 
    engineer.add_op("impl", day, duration, [feature.name, b])
    daily_schedule.append((day + duration, b))
    binaries[b].working_on+=1
    
    return duration

def move_service(e: int, day: int, s: int, dst: int) -> int:
    assert(s >= 0 and s < S)
    assert(day >= 0 and  day < L)
    assert(dst >= 0 and dst < B)
    assert(e >= 0 and e < G)
    engineer = engineers[e]
    service = services[s]
    assert(service_to_binaries[service.name] != dst)
    src = service.binary
    duration  = max(len(binaries[src].services), len(binaries[dst].services))
    service_to_binaries[service.name] = dst
    binaries[src].services.remove(s)
    binaries[dst].services.append(s)
    binaries[src].locked = True
    binaries[dst].locked = True
    engineer.add_op("move", day, duration, [s, src, dst])
    return duration

def new_binary(e: Engineer, day: int)-> int:
    assert(day >=0 and day < L)
    binaries.append(Binary(len(binaries), [], 0, False))
    e.add_op("new", day, N, [])
    return N

def wait(e: Engineer, day: int, duration: int) -> int:
    assert(day >=0 and day < L)
    e.add_op("wait", day, duration, [])
    return duration

for day in range(L):
    while len(daily_schedule) > 0 and daily_schedule[-1][0] <= day:
        # migrazione
        if len(daily_schedule[-1]) == 3:
            b1 = daily_schedule[-1][1]
            b2 = daily_schedule[-1][2]
            binaries[b1].locked = False 
            binaries[b2].locked = False
        # implementazione feature
        elif len(daily_schedule[-1]) == 2:
            bid: int = daily_schedule[-1][1] 
            binaries[bid].working_on -=1
        else:
            raise Exception("unknown schedule event", daily_schedule[-1])
        daily_schedule.pop()
    
    for engineer in engineers:
        if engineer.avail <= day:
          if random.random() > .5:
            s: Service = random.choice(services)
            b: Binary = random.choice(binaries)
            while b.id == s.binary:
                b: Binary = random.choice(binaries)
            move_service(engineer.id, day, s.id, b.id)
          else:
            f: Feature = random.choice(features)
            b: Binary = random.choice(binaries)
            implement_feature(engineer.id, day, f.idx, b.id)

working_engineers: List[Engineer] = [engineer for engineer in engineers if len(engineer.ops) > 0]

ofname = pathlib.Path(fname).with_suffix(".out")
with open(ofname, "w") as file:
    file.write(str(len(working_engineers)) + "\n")
    for engineer in working_engineers:
        file.write(str(engineer.id)+ "\n")
        for op in engineer.ops:
            file.write(op[0] +" " +  " ".join([str (k) for k in op[1]]) + "\n")

# print(engineers, binaries, service_to_binaries, features, services, sep="\n")
print("score = ", score)
completed_features = [f for f in features if f.completed and f.day_completed < L]
print("completed", len(completed_features), "features")
# record 158.421.404

ValueError: list.remove(x): x not in list